In [0]:
pyspark
from aaintdatapipeline.core.fs.device import meta_bucket
import collections
from copy import deepcopy
from aaintdatapipeline.core.pipeline.type_ import DataType, EventType, ManipulationType
from aaintdatapipeline.core.utils.identifier import date_time, type_
from aaintdatapipeline.core.utils import avro_, json_
from aaintdatapipeline.core.fs.router import EventRouter
from aaintdatapipeline.core.fs.device import meta_bucket


s3_path = "s3://b2b-prod-int-data-pipeline-metastore/meta/events/app-int.city-level-usage.v1/load/insert/220190808061256077/aidp35adf9791115505117fc027362ff153d.avro"



def _get_params_from_avro():
    # prefix = EventRouter(urn).logic_key()
    # try:
    # key = meta_bucket().all(prefix)[0]
    # print "key in urn"
    # print key
    s3_prefix = "meta/events/app-int.city-level-usage.v1/load/insert/220190808061256077/"
    key = meta_bucket().all(s3_prefix)[0]
    data = meta_bucket().get(key)
    encode_parms_ = avro_.read(byte_array=data)[0]['params']
    parms_in_dict = json_.loads(encode_parms_)['py/collections.namedtuple']
    return collections.namedtuple("parms", parms_in_dict['fields']) \
        ._make(parms_in_dict['values'])
    # except:
    #     return  None

'workflow_description' in dir(_get_params_from_avro())


In [0]:
%%sh

#PGPASSWORD=2mHdFW6%#REu psql -h 10.2.6.27 -p 5433 -Uapp_bdp_usage_qa -d mkt_est << EOF
#select max(id) from mkt.s3_path_creative_id_map where update_time < '2019-05-01';


# PGPASSWORD=2mHdFW6%#REu psql -h 10.2.6.213 -p 5432 -Uapp_bdp_usage_qa -d mkt_est << EOF
# \l


#PG_MKT_EST_P0_HOSTS = [(‘10.2.6.27’, 5433)]
#PG_MKT_EST_P1_HOSTS = [(‘10.2.6.245’, 5434)]
#PG_MKT_EST_P2_HOSTS = [(‘10.2.6.42’, 5435)]
#PG_MKT_EST_P3_HOSTS = [(‘10.2.6.148’, 5436)]
#PG_MKT_EST_P4_HOSTS = [(‘10.2.6.49’, 5437)]
#PG_MKT_EST_P5_HOSTS = [(‘10.2.6.52’, 5438)]
#PG_MKT_EST_P6_HOSTS = [(‘10.2.6.187’, 5439)]
#PG_MKT_EST_P7_HOSTS = [(‘10.2.6.254’, 5440)]

PGPASSWORD=2mHdFW6%#REu psql -h 10.2.6.213 -p 5432 -Uapp_bdp_usage_qa -d mkt_est << EOF 
SELECT sum(creative_id)
FROM plproxy.execute_select_nestloop($proxy$
	SELECT count(creative_id)
	FROM mkt.meta_creative_1001
$proxy$) t (creative_id BIGINT)

# PGPASSWORD=2mHdFW6%#REu psql -h 10.2.6.27 -p 5433 -Uapp_bdp_usage_qa -d mkt_est << EOF
# SELECT count(creative_id) FROM mkt.meta_creative_1001





In [0]:

import datetime
import psycopg2
from contextlib import closing

# PGPASSWORD=2mHdFW6%#REu psql -h 10.2.6.213 -p 5432 -Uapp_bdp_usage_qa -d mkt_est << EOF
dbinfo = {
    'NAME': 'mkt_est',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': '10.2.6.213',
    'PORT': 5432,
    
}

dbinfo = {
    'NAME': 'mkt_est',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': '10.2.6.27',
    'PORT': 5433,
    
}
# dsn = "dbname='{dbname}' user='{user}' host='{host}' port='{port}' password='{passwd}'".format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
# print dsn
conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])

# psycopg2.connect(database="postgres",
#                                     user=self.user,
#                                     password=self.passwd)
# conn = psycopg2.connect(dsn)
def select(sql):
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt
    
    
def get_db_count(key, schema, table):
    sql = """
         SELECT sum({key}) FROM plproxy.execute_select
         ($$
              SELECT count({key}) FROM {schema}.{table}
         $$) t ({key} REAL)
    """.format(
        key=key,
        schema=schema,
        table=table
    )
    select(sql)
    
get_db_count('creative_id', 'mkt', 'weekly_creative_detail_estimate_1001_1' )


In [0]:

import datetime
import psycopg2
from contextlib import closing

dbinfo = {
    'NAME': 'mkt_est',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': '10.2.6.27',
    'PORT': 5433,
    
}

conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])

def select(sql):
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt
    
def get_db_count(key, schema, table):
    sql = """
        select max(id) from mkt.s3_path_creative_id_map where update_time < '2019-05-01';
    """.format(
        key=key,
        schema=schema,
        table=table
    )
    return select(sql)
    
rows, _ = get_db_count('creative_id', 'mkt', 'weekly_creative_detail_estimate_1001_1')
print rows[0][0]

table = "mkt.meta_creative_{device_id}_{store_id}_{network_id}"






In [0]:

from itertools import product
import datetime
import psycopg2
from contextlib import closing
import time

GP_COUNTRY_CODE = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95, 1000]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = [1001, 1002] # 1000
IOS_DEVICE_ID = [2001, 2002] # 2000
NETWORK = [1, 203, 324, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 114, 116, 142, 311, 121, 316, 325]
DATE = ['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904']

dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27', 'PORT': 5433}
dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]

time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
log_filename = '/tmp/db_{time}.log'.format(time=time)
print log_filename

def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def get_device_store_list():
    sql_list = []
    for device_id, store_id  in product(GP_DEVICE_ID, GP_COUNTRY_CODE):
        sql_list.append((device_id, store_id))
    for device_id, store_id in product(IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        sql_list.append((device_id, store_id))
    print len(sql_list)
    return sql_list

def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str)

for device_id,store_id in get_device_store_list():
    table_name = 'meta_creative'
    # SQL RESULT
    result = 0
    sql_str = "{device_id}_{store_id}".format(device_id=device_id, store_id=store_id)
    for host, port in dblist:
        dbinfo['HOST'] = host
        dbinfo['PORT'] = port
        try:
            sql =  "SELECT count(distinct(device_id, store_id, network_id, creative_id, ad_app_id, first_seen, last_seen)) FROM mkt.meta_creative_{sql_str} WHERE creative_id <= 90100006099406".format(sql_str=sql_str)
            # print sql # debug
            r, _ = select(sql)
            result += r[0][0]
            # print result # debug
        except Exception, e:
            error_result=e.message.replace('\n', '\\n')
            if "does not exist" in error_result:
                error_result = -1
    # S3 RESULT
    s3_result = 0
    s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-meta_creative.v1/dimension/device_id={device_id}/store_id={store_id}/".format(device_id=device_id, store_id=store_id)
    try:
        s3_result = spark.read.parquet(s3_path).count()
    except Exception, e:
        error_s3_result = -1

    result_str = "mkt.meta_creative_{},{},{}\n".format(sql_str, result or error_result,s3_result or error_s3_result)
    write_to_file(log_filename, result_str)
    # raise Exception
    




In [0]:
%%sh

cat /tmp/db_2019-08-19_12-50-45.log



In [0]:

from itertools import product
import datetime
import psycopg2
from contextlib import closing
import time

GP_COUNTRY_CODE = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95, 1000]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = [1001, 1002] # 1000
IOS_DEVICE_ID = [2001, 2002] # 2000
GRANULARIY = ["weekly", "monthly"]
NETWORK = [1, 203, 324, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 114, 116, 142, 311, 121, 316, 325]
DATE = ['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904']

dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27', 'PORT': 5433}
dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]

time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
log_filename = '/tmp/db_{time}.log'.format(time=time)
print log_filename

def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def get_device_store_list():
    product_list = []
    for device_id, store_id  in product(GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    for device_id, store_id in product(IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    print len(product_list)
    return product_list

def get_granularity_device_store_list():
    product_list = []
    for granularity, device_id, store_id  in product(GRANULARIY, GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    for granularity, device_id, store_id in product(GRANULARIY, IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    print len(product_list)
    return product_list


def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str)

for granularity,device_id,store_id in get_granularity_device_store_list():
    table_name = 'ad_app_estimate'
    # SQL RESULT
    result = 0
    sql_str = "{granularity}_{table_name}_{device_id}_{store_id}".format(table_name=table_name, device_id=device_id, store_id=store_id, granularity=granularity)
    for host, port in dblist:
        dbinfo['HOST'] = host
        dbinfo['PORT'] = port
        try:
            sql =  "SELECT count(date) FROM mkt.{sql_str} WHERE date < '2019-05-01'".format(sql_str=sql_str)
            # print sql # debug
            r, _ = select(sql)
            result += r[0][0]
            # print result # debug
        except Exception, e:
            # print e.message
            error_result=e.message.replace('\n', '\\n')
            if "does not exist" in error_result:
                error_result = -1
    # S3 RESULT
    s3_result = 0
    s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-ad_app_estimate.v1/fact/granularity={granularity}/device_id={device_id}/store_id={store_id}/part-{granularity}-{device_id}-{store_id}.c000.gz.parquet".format(granularity=granularity, device_id=device_id, store_id=store_id)
    try:
        s3_result = spark.read.parquet(s3_path).count()
    except Exception, e:
        error_s3_result = -1

    result_str = "{},{},{}\n".format(sql_str, result or error_result,s3_result or error_s3_result)
    write_to_file(log_filename, result_str)
    # raise Exception

In [0]:

from itertools import product
import datetime
import psycopg2
from contextlib import closing
import time

GP_COUNTRY_CODE = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95, 1000]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = [1001, 1002] # 1000
IOS_DEVICE_ID = [2001, 2002] # 2000
GRANULARIY = ["weekly", "monthly"]
NETWORK = [1, 203, 324, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 114, 116, 142, 311, 121, 316, 325]
DATE = ['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904']

dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27', 'PORT': 5433}
dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]

time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
log_filename = '/tmp/db_{time}.log'.format(time=time)
print log_filename

def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def get_device_store_list():
    product_list = []
    for device_id, store_id  in product(GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    for device_id, store_id in product(IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    print len(product_list)
    return product_list

def get_granularity_device_store_list():
    product_list = []
    for granularity, device_id, store_id  in product(GRANULARIY, GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    for granularity, device_id, store_id in product(GRANULARIY, IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    print len(product_list)
    return product_list


def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str)

for granularity,device_id,store_id in get_granularity_device_store_list():
    table_name = 'ad_app_detail_estimate'
    # SQL RESULT
    result = 0
    sql_str = "{granularity}_{table_name}_{device_id}_{store_id}".format(table_name=table_name, device_id=device_id, store_id=store_id, granularity=granularity)
    for host, port in dblist:
        dbinfo['HOST'] = host
        dbinfo['PORT'] = port
        try:
            sql =  "SELECT count(date) FROM mkt.{sql_str} WHERE date < '2019-05-01'".format(sql_str=sql_str)
            # print sql # debug
            r, _ = select(sql)
            result += r[0][0]
            # print result # debug
        except Exception, e:
            # print e.message
            error_result=e.message.replace('\n', '\\n')
            if "does not exist" in error_result:
                error_result = -1
    # S3 RESULT
    s3_result = 0
    s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-{table_name}.v1/fact/granularity={granularity}/device_id={device_id}/store_id={store_id}/part-{granularity}-{device_id}-{store_id}.c000.gz.parquet".format(table_name=table_name,granularity=granularity, device_id=device_id, store_id=store_id)
    try:
        s3_result = spark.read.parquet(s3_path).count()
    except Exception, e:
        error_s3_result = -1

    result_str = "{},{},{}\n".format(sql_str, result or error_result,s3_result or error_s3_result)
    write_to_file(log_filename, result_str)
    # raise Exception

In [0]:


from itertools import product
import datetime
import psycopg2
from contextlib import closing
import time

GP_COUNTRY_CODE = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95, 1000]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = [1001, 1002] # 1000
IOS_DEVICE_ID = [2001, 2002] # 2000
GRANULARIY = ["weekly", "monthly"]
NETWORK = [1, 203, 324, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 114, 116, 142, 311, 121, 316, 325]
DATE = ['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904']

dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27', 'PORT': 5433}
dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]

time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
log_filename = '/tmp/db_{time}.log'.format(time=time)
print log_filename

def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def get_device_store_list():
    product_list = []
    for device_id, store_id  in product(GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    for device_id, store_id in product(IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    print len(product_list)
    return product_list

def get_granularity_device_store_list():
    product_list = []
    for granularity, device_id, store_id  in product(GRANULARIY, GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    for granularity, device_id, store_id in product(GRANULARIY, IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    print len(product_list)
    return product_list


def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str)

for granularity,device_id,store_id in get_granularity_device_store_list():
    table_name = 'ad_app_pub_estimate'
    # SQL RESULT
    result = 0
    sql_str = "{granularity}_{table_name}_{device_id}_{store_id}".format(table_name=table_name, device_id=device_id, store_id=store_id, granularity=granularity)
    for host, port in dblist:
        dbinfo['HOST'] = host
        dbinfo['PORT'] = port
        try:
            sql =  "SELECT count(date) FROM mkt.{sql_str} WHERE date < '2019-05-01'".format(sql_str=sql_str)
            # print sql # debug
            r, _ = select(sql)
            result += r[0][0]
            # print result # debug
        except Exception, e:
            # print e.message
            error_result=e.message.replace('\n', '\\n')
            if "does not exist" in error_result:
                error_result = -1
    # S3 RESULT
    s3_result = 0
    s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-{table_name}.v1/fact/granularity={granularity}/device_id={device_id}/store_id={store_id}/part-{granularity}-{device_id}-{store_id}.c000.gz.parquet".format(table_name=table_name,granularity=granularity, device_id=device_id, store_id=store_id)
    try:
        s3_result = spark.read.parquet(s3_path).count()
    except Exception, e:
        error_s3_result = -1

    result_str = "{},{},{}\n".format(sql_str, result or error_result,s3_result or error_s3_result)
    write_to_file(log_filename, result_str)
    # raise Exception

In [0]:


from itertools import product
import datetime
import psycopg2
from contextlib import closing
import time

GP_COUNTRY_CODE = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95, 1000]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = [1001, 1002] # 1000
IOS_DEVICE_ID = [2001, 2002] # 2000
GRANULARIY = ["weekly", "monthly"]
NETWORK = [1, 203, 324, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 114, 116, 142, 311, 121, 316, 325]
DATE = ['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904']

dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27', 'PORT': 5433}
dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]

time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
log_filename = '/tmp/db_{time}.log'.format(time=time)
print log_filename

def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def get_device_store_list():
    product_list = []
    for device_id, store_id  in product(GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    for device_id, store_id in product(IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    print len(product_list)
    return product_list

def get_granularity_device_store_list():
    product_list = []
    for granularity, device_id, store_id  in product(GRANULARIY, GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    for granularity, device_id, store_id in product(GRANULARIY, IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    print len(product_list)
    return product_list


def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str)

for granularity,device_id,store_id in get_granularity_device_store_list():
    table_name = 'pub_app_estimate'
    # SQL RESULT
    result = 0
    sql_str = "{granularity}_{table_name}_{device_id}_{store_id}".format(table_name=table_name, device_id=device_id, store_id=store_id, granularity=granularity)
    for host, port in dblist:
        dbinfo['HOST'] = host
        dbinfo['PORT'] = port
        try:
            sql =  "SELECT count(date) FROM mkt.{sql_str} WHERE date < '2019-05-01'".format(sql_str=sql_str)
            # print sql # debug
            r, _ = select(sql)
            result += r[0][0]
            # print result # debug
        except Exception, e:
            # print e.message
            error_result=e.message.replace('\n', '\\n')
            if "does not exist" in error_result:
                error_result = -1
    # S3 RESULT
    s3_result = 0
    s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-{table_name}.v1/fact/granularity={granularity}/device_id={device_id}/store_id={store_id}/part-{granularity}-{device_id}-{store_id}.c000.gz.parquet".format(table_name=table_name,granularity=granularity, device_id=device_id, store_id=store_id)
    try:
        s3_result = spark.read.parquet(s3_path).count()
    except Exception, e:
        error_s3_result = -1

    result_str = "{},{},{}\n".format(sql_str, result or error_result,s3_result or error_s3_result)
    write_to_file(log_filename, result_str)
    # raise Exception

In [0]:


from itertools import product
import datetime
import psycopg2
from contextlib import closing
import time

GP_COUNTRY_CODE = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95, 1000]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = [1001, 1002] # 1000
IOS_DEVICE_ID = [2001, 2002] # 2000
GRANULARIY = ["weekly", "monthly"]
NETWORK = [1, 203, 324, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 114, 116, 142, 311, 121, 316, 325]
DATE = ['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904']

dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27', 'PORT': 5433}
dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]

time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
log_filename = '/tmp/db_{time}.log'.format(time=time)
print log_filename

def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def get_device_store_list():
    product_list = []
    for device_id, store_id  in product(GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    for device_id, store_id in product(IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    print len(product_list)
    return product_list

def get_granularity_device_store_list():
    product_list = []
    for granularity, device_id, store_id  in product(GRANULARIY, GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    for granularity, device_id, store_id in product(GRANULARIY, IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    print len(product_list)
    return product_list


def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str)

for granularity,device_id,store_id in get_granularity_device_store_list():
    table_name = 'pub_app_detail_estimate'
    # SQL RESULT
    result = 0
    sql_str = "{granularity}_{table_name}_{device_id}_{store_id}".format(table_name=table_name, device_id=device_id, store_id=store_id, granularity=granularity)
    for host, port in dblist:
        dbinfo['HOST'] = host
        dbinfo['PORT'] = port
        try:
            sql =  "SELECT count(date) FROM mkt.{sql_str} WHERE date < '2019-05-01'".format(sql_str=sql_str)
            # print sql # debug
            r, _ = select(sql)
            result += r[0][0]
            # print result # debug
        except Exception, e:
            # print e.message
            error_result=e.message.replace('\n', '\\n')
            if "does not exist" in error_result:
                error_result = -1
    # S3 RESULT
    s3_result = 0
    s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-{table_name}.v1/fact/granularity={granularity}/device_id={device_id}/store_id={store_id}/part-{granularity}-{device_id}-{store_id}.c000.gz.parquet".format(table_name=table_name,granularity=granularity, device_id=device_id, store_id=store_id)
    try:
        s3_result = spark.read.parquet(s3_path).count()
    except Exception, e:
        error_s3_result = -1

    result_str = "{},{},{}\n".format(sql_str, result or error_result,s3_result or error_s3_result)
    write_to_file(log_filename, result_str)
    # raise Exception

In [0]:


from itertools import product
import datetime
import psycopg2
from contextlib import closing
import time

GP_COUNTRY_CODE = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95, 1000]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = [1001, 1002] # 1000
IOS_DEVICE_ID = [2001, 2002] # 2000
GRANULARIY = ["weekly", "monthly"]
NETWORK = [1, 203, 324, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 114, 116, 142, 311, 121, 316, 325]
DATE = ['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904']

dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27', 'PORT': 5433}
dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]

time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
log_filename = '/tmp/db_{time}.log'.format(time=time)
print log_filename

def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def get_device_store_list():
    product_list = []
    for device_id, store_id  in product(GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    for device_id, store_id in product(IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    print len(product_list)
    return product_list

def get_granularity_device_store_list():
    product_list = []
    for granularity, device_id, store_id  in product(GRANULARIY, GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    for granularity, device_id, store_id in product(GRANULARIY, IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    print len(product_list)
    return product_list


def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str)

for granularity,device_id,store_id in get_granularity_device_store_list():
    table_name = 'creative_category'
    # SQL RESULT
    result = 0
    sql_str = "{granularity}_{table_name}_{device_id}_{store_id}".format(table_name=table_name, device_id=device_id, store_id=store_id, granularity=granularity)
    for host, port in dblist:
        dbinfo['HOST'] = host
        dbinfo['PORT'] = port
        try:
            sql =  "SELECT count(date) FROM mkt.{sql_str} WHERE date < '2019-05-01'".format(sql_str=sql_str)
            # print sql # debug
            r, _ = select(sql)
            result += r[0][0]
            # print result # debug
        except Exception, e:
            # print e.message
            error_result=e.message.replace('\n', '\\n')
            if "does not exist" in error_result:
                error_result = -1
    # S3 RESULT
    s3_result = 0
    s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-{table_name}.v1/fact/granularity={granularity}/device_id={device_id}/store_id={store_id}/part-{granularity}-{device_id}-{store_id}.c000.gz.parquet".format(table_name=table_name,granularity=granularity, device_id=device_id, store_id=store_id)
    try:
        s3_result = spark.read.parquet(s3_path).count()
    except Exception, e:
        error_s3_result = -1

    result_str = "{},{},{}\n".format(sql_str, result or error_result,s3_result or error_s3_result)
    write_to_file(log_filename, result_str)
    # raise Exception

In [0]:


from itertools import product
import datetime
import psycopg2
from contextlib import closing
import time

GP_COUNTRY_CODE = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95, 1000]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = [1001, 1002] # 1000
IOS_DEVICE_ID = [2001, 2002] # 2000
GRANULARIY = ["weekly", "monthly"]
NETWORK = [1, 203, 324, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 114, 116, 142, 311, 121, 316, 325]
DATE = ['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904']

dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27', 'PORT': 5433}
dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]

time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
log_filename = '/tmp/db_{time}.log'.format(time=time)
print log_filename

def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def get_device_store_list():
    product_list = []
    for device_id, store_id  in product(GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    for device_id, store_id in product(IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    print len(product_list)
    return product_list

def get_granularity_device_store_list():
    product_list = []
    for granularity, device_id, store_id  in product(GRANULARIY, GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    for granularity, device_id, store_id in product(GRANULARIY, IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    print len(product_list)
    return product_list


def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str)

for granularity,device_id,store_id in get_granularity_device_store_list():
    table_name = 'ad_app_category'
    # SQL RESULT
    result = 0
    sql_str = "{granularity}_{table_name}_{device_id}_{store_id}".format(table_name=table_name, device_id=device_id, store_id=store_id, granularity=granularity)
    for host, port in dblist:
        dbinfo['HOST'] = host
        dbinfo['PORT'] = port
        try:
            sql =  "SELECT count(date) FROM mkt.{sql_str} WHERE date < '2019-05-01'".format(sql_str=sql_str)
            # print sql # debug
            r, _ = select(sql)
            result += r[0][0]
            # print result # debug
        except Exception, e:
            # print e.message
            error_result=e.message.replace('\n', '\\n')
            if "does not exist" in error_result:
                error_result = -1
    # S3 RESULT
    s3_result = 0
    s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-{table_name}.v1/fact/granularity={granularity}/device_id={device_id}/store_id={store_id}/part-{granularity}-{device_id}-{store_id}.c000.gz.parquet".format(table_name=table_name,granularity=granularity, device_id=device_id, store_id=store_id)
    try:
        s3_result = spark.read.parquet(s3_path).count()
    except Exception, e:
        error_s3_result = -1

    result_str = "{},{},{}\n".format(sql_str, result or error_result,s3_result or error_s3_result)
    write_to_file(log_filename, result_str)
    # raise Exception

In [0]:


from itertools import product
import datetime
import psycopg2
from contextlib import closing
import time

GP_COUNTRY_CODE = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95, 1000]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = [1001, 1002] # 1000
IOS_DEVICE_ID = [2001, 2002] # 2000
GRANULARIY = ["weekly", "monthly"]
NETWORK = [1, 203, 324, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 114, 116, 142, 311, 121, 316, 325]
DATE = ['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904']

dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27', 'PORT': 5433}
dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]

time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
log_filename = '/tmp/db_{time}.log'.format(time=time)
print log_filename

def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def get_device_store_list():
    product_list = []
    for device_id, store_id  in product(GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    for device_id, store_id in product(IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    print len(product_list)
    return product_list

def get_granularity_device_store_list():
    product_list = []
    for granularity, device_id, store_id  in product(GRANULARIY, GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    for granularity, device_id, store_id in product(GRANULARIY, IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    print len(product_list)
    return product_list


def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str)

for granularity,device_id,store_id in get_granularity_device_store_list():
    table_name = 'pub_app_category'
    # SQL RESULT
    result = 0
    sql_str = "{granularity}_{table_name}_{device_id}_{store_id}".format(table_name=table_name, device_id=device_id, store_id=store_id, granularity=granularity)
    for host, port in dblist:
        dbinfo['HOST'] = host
        dbinfo['PORT'] = port
        try:
            sql =  "SELECT count(date) FROM mkt.{sql_str} WHERE date < '2019-05-01'".format(sql_str=sql_str)
            # print sql # debug
            r, _ = select(sql)
            result += r[0][0]
            # print result # debug
        except Exception, e:
            # print e.message
            error_result=e.message.replace('\n', '\\n')
            if "does not exist" in error_result:
                error_result = -1
    # S3 RESULT
    s3_result = 0
    s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-{table_name}.v1/fact/granularity={granularity}/device_id={device_id}/store_id={store_id}/part-{granularity}-{device_id}-{store_id}.c000.gz.parquet".format(table_name=table_name,granularity=granularity, device_id=device_id, store_id=store_id)
    try:
        s3_result = spark.read.parquet(s3_path).count()
    except Exception, e:
        error_s3_result = -1

    result_str = "{},{},{}\n".format(sql_str, result or error_result,s3_result or error_s3_result)
    write_to_file(log_filename, result_str)
    # raise Exception

In [0]:


from itertools import product
import datetime
import psycopg2
from contextlib import closing
import time

GP_COUNTRY_CODE = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95, 1000]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = [1000] # 1000
IOS_DEVICE_ID = [2000] # 2000
GRANULARIY = ["monthly","weekly"]
NETWORK = [1, 203, 324, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 114, 116, 142, 311, 121, 316, 325]
DATE = ['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904']

dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27', 'PORT': 5433}
dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]

time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
log_filename = '/tmp/db_{time}.log'.format(time=time)
print log_filename

def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def get_device_store_list():
    product_list = []
    for device_id, store_id  in product(GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    for device_id, store_id in product(IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    print len(product_list)
    return product_list

def get_granularity_device_store_list():
    product_list = []
    for granularity, device_id, store_id  in product(GRANULARIY, GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    for granularity, device_id, store_id in product(GRANULARIY, IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    print len(product_list)
    return product_list

def get_granularity_device_list():
    product_list = []
    for granularity, device_id  in product(GRANULARIY, GP_DEVICE_ID):
        product_list.append((granularity, device_id))
    for granularity, device_id in product(GRANULARIY, IOS_DEVICE_ID):
        product_list.append((granularity, device_id))
    print len(product_list)
    return product_list


def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str)

for granularity,device_id in get_granularity_device_list():
    table_name = 'adpo_estimate'
    # SQL RESULT
    result = 0
    sql_str = "{granularity}_{table_name}_{device_id}".format(table_name=table_name, device_id=device_id, granularity=granularity)
    for host, port in dblist:
        dbinfo['HOST'] = host
        dbinfo['PORT'] = port
        try:
            sql =  "SELECT count(date) FROM mkt.{sql_str} WHERE date < '2019-05-01'".format(sql_str=sql_str)
            # print sql # debug
            r, _ = select(sql)
            result += r[0][0]
            # print result # debug
        except Exception, e:
            # print e.message
            error_result=e.message.replace('\n', '\\n')
            if "does not exist" in error_result:
                error_result = -1
    # S3 RESULT
    s3_result = 0
    s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-{table_name}.v1/fact/granularity={granularity}/device_id={device_id}/".format(table_name=table_name,granularity=granularity, device_id=device_id)
    try:
        s3_result = spark.read.parquet(s3_path).count()
    except Exception, e:
        error_s3_result = -1

    result_str = "{},{},{}\n".format(sql_str, result or error_result,s3_result or error_s3_result)
    write_to_file(log_filename, result_str)
    # raise Exception

In [0]:


from itertools import product
import datetime
import psycopg2
from contextlib import closing
import time

GP_COUNTRY_CODE = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95, 1000]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = [1000] # 1000
IOS_DEVICE_ID = [2000] # 2000
GRANULARIY = ["monthly","weekly"]
NETWORK = [1, 203, 324, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 114, 116, 142, 311, 121, 316, 325]
DATE = ['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904']

dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27', 'PORT': 5433}
dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]

time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
log_filename = '/tmp/db_{time}.log'.format(time=time)
print log_filename

def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def get_device_store_list():
    product_list = []
    for device_id, store_id  in product(GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    for device_id, store_id in product(IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    print len(product_list)
    return product_list

def get_granularity_device_store_list():
    product_list = []
    for granularity, device_id, store_id  in product(GRANULARIY, GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    for granularity, device_id, store_id in product(GRANULARIY, IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    print len(product_list)
    return product_list

def get_granularity_device_list():
    product_list = []
    for granularity, device_id  in product(GRANULARIY, GP_DEVICE_ID):
        product_list.append((granularity, device_id))
    for granularity, device_id in product(GRANULARIY, IOS_DEVICE_ID):
        product_list.append((granularity, device_id))
    print len(product_list)
    return product_list


def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str)

for granularity,device_id in get_granularity_device_list():
    table_name = 'adpo_detail_estimate'
    # SQL RESULT
    result = 0
    sql_str = "{granularity}_{table_name}_{device_id}".format(table_name=table_name, device_id=device_id, granularity=granularity)
    for host, port in dblist:
        dbinfo['HOST'] = host
        dbinfo['PORT'] = port
        try:
            sql =  "SELECT count(date) FROM mkt.{sql_str} WHERE date < '2019-05-01'".format(sql_str=sql_str)
            # print sql # debug
            r, _ = select(sql)
            result += r[0][0]
            # print result # debug
        except Exception, e:
            # print e.message
            error_result=e.message.replace('\n', '\\n')
            if "does not exist" in error_result:
                error_result = -1
    # S3 RESULT
    s3_result = 0
    s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-{table_name}.v1/fact/granularity={granularity}/device_id={device_id}/".format(table_name=table_name,granularity=granularity, device_id=device_id)
    try:
        s3_result = spark.read.parquet(s3_path).count()
    except Exception, e:
        error_s3_result = -1

    result_str = "{},{},{}\n".format(sql_str, result or error_result,s3_result or error_s3_result)
    write_to_file(log_filename, result_str)
    # raise Exception

In [0]:
%%sh
aws s3 ls --recursive s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-adpo_estimate.v1/fact/ 
cat /tmp/db_2019-08-19_02-28-56.log





In [0]:


# print spark.read.parquet('s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-meta_creative.v1/dimension/device_id=1001/store_id=35/part-1001-35.c000.gz.parquet').
# print spark.read.parquet('s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-creative_extension_map.v1/dimension/_partition=_default_partition/part-_default_partition.c000.gz.parquet').count()
df = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-s3_path_creative_id_map.v1/dimension/_partition=_default_partition/part-_default_partition.c000.gz.parquet')

types = [f for f in df.columns]
print types
print dir(df)


In [0]:

import psycopg2
from contextlib import closing

dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]
def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

# SQL RESULT
for host, port in dblist:
    dbinfo['HOST'] = host
    dbinfo['PORT'] = port
    try:
        sql =  "SELECT max(last_seen) FROM mkt.meta_creative_1002_38 WHERE creative_id <= 90100006099406".format(sql_str=sql_str)
        # print sql # debug
        r, _ = select(sql)
        print r
    except Exception, e:
       print e.message
       
# # S3 RESULT
# s3_result = 0
# s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-meta_creative.v1/dimension/device_id={device_id}/store_id={store_id}/".format(device_id=device_id, store_id=store_id)
# try:
#     s3_result = spark.read.parquet(s3_path).count()
# except Exception, e:
#     error_s3_result = -1

# result_str = "mkt.meta_creative_{},{},{}\n".format(sql_str, result or error_result,s3_result or error_s3_result)
# write_to_file(log_filename, result_str)
# raise Exception
    




In [0]:
%%sh

cat /tmp/db_2019-08-19_02-47-54.log







In [0]:


from itertools import product
import datetime
import psycopg2
from contextlib import closing
import time

GP_COUNTRY_CODE = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95, 1000]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = [1000, 1001, 1002]
IOS_DEVICE_ID = [2000, 2001, 2002]
GRANULARIY = ["weekly", "monthly"]
NETWORK = [1, 203, 324, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 114, 116, 142, 311, 121, 316, 325]
DATE = ['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904']

dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27', 'PORT': 5433}
dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]

time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
log_filename = '/tmp/db_{time}.log'.format(time=time)
print log_filename

def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def get_device_store_list():
    product_list = []
    for device_id, store_id  in product(GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    for device_id, store_id in product(IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((device_id, store_id))
    print len(product_list)
    return product_list

def get_granularity_device_store_list():
    product_list = []
    for granularity, device_id, store_id  in product(GRANULARIY, GP_DEVICE_ID, GP_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    for granularity, device_id, store_id in product(GRANULARIY, IOS_DEVICE_ID, IOS_COUNTRY_CODE):
        product_list.append((granularity, device_id, store_id))
    print len(product_list)
    return product_list


def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str)

for granularity,device_id,store_id in get_granularity_device_store_list():
    table_name = 'ad_app_category'
    # SQL RESULT
    result = 0
    sql_str = "{granularity}_{table_name}_{device_id}_{store_id}".format(table_name=table_name, device_id=device_id, store_id=store_id, granularity=granularity)
    for host, port in dblist:
        dbinfo['HOST'] = host
        dbinfo['PORT'] = port
        try:
            sql =  "SELECT count(date) FROM mkt.{sql_str} WHERE date < '2019-05-01'".format(sql_str=sql_str)
            # print sql # debug
            r, _ = select(sql)
            result += r[0][0]
            # print result # debug
        except Exception, e:
            # print e.message
            error_result=e.message.replace('\n', '\\n')
            if "does not exist" in error_result:
                error_result = -1
    # S3 RESULT
    s3_result = 0
    s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-{table_name}.v1/fact/granularity={granularity}/device_id={device_id}/store_id={store_id}/part-{granularity}-{device_id}-{store_id}.c000.gz.parquet".format(table_name=table_name,granularity=granularity, device_id=device_id, store_id=store_id)
    try:
        s3_result = spark.read.parquet(s3_path).count()
    except Exception, e:
        error_s3_result = -1

    result_str = "{},{},{}\n".format(sql_str, result or error_result,s3_result or error_s3_result)
    write_to_file(log_filename, result_str)
    # raise Exception